# 01. Data Collection

**Purpose:** This notebook demonstrates how to collect real-time game data using our custom `DataCollector` class. It fetches data from Steam (player counts, details), Twitch (viewership), and external sources like Google Trends, Reddit, Twitter, and YouTube.

**Why This Matters:** This automated collection process provides fresh, multi-faceted data crucial for building our time series dataset and ultimately, the game popularity prediction model.

**What to Expect:** After running this notebook, you will:
1. Successfully collect current data from multiple APIs.
2. Save the combined data in a structured format (compressed CSV).
3. Understand the enhanced data collection workflow.
4. Have the first data point for building a historical dataset.

## 1. Setup and Configuration

**Purpose:** Import necessary libraries and configure the environment.

**Why:** Ensures the `DataCollector` and its dependencies can be found and used correctly.

In [1]:
# Imports and Setup
import sys
import os
import pandas as pd
from datetime import datetime

# Add src directory to path to import modules
# Assumes notebook is run from the 'notebooks' directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import our custom modules
from src.data_collector import DataCollector
# from src.utils import configure_plotting # Optional: if plotting is needed here

# Configure plotting (optional)
# configure_plotting()

# Display pandas DataFrames nicely
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Display current time for reference
print(f"Notebook Execution Started: {datetime.now()}")

Notebook Execution Started: 2025-05-06 22:00:33.338323


## 2. Initialize Data Collector

**Purpose:** Create an instance of the `DataCollector`.

**Why:** The collector manages API connections, game lists, and data storage. It automatically loads API keys from the `.env` file in the project root.

**Expected Output:** Confirmation of initialization and the number of games being tracked.

In [2]:
# Initialize the collector
# It will use the default game lists defined within the class
# and look for API keys in the .env file in the project root
# Ensure the data_dir path is correct relative to the notebook location
collector = DataCollector(data_dir="../data")

# Optionally, view the game IDs being tracked
all_game_ids = collector.get_all_game_ids()
print(f"Tracking {len(all_game_ids)} unique game IDs across categories.")
# print(collector.game_categories) # Uncomment to see the full category list

TwitchAPIConnector initialized
[ExternalDataCollector] Attempting to load .env file...
[ExternalDataCollector] .env file found and loaded: True
Reddit client initialized successfully (read-only mode).
Error initializing Twitter client: BaseClient.__init__() got an unexpected keyword argument 'timeout'
[ExternalDataCollector._init_pytrends] Detected pytrends library version: unknown (attribute missing)
         Consider checking your pytrends installation (target version 4.7.0+ for full support).
Pytrends client initialized successfully (fallback, with requests_args for SSL verification).
ExternalDataCollector initialized
Tracking 20 unique game IDs across categories.


c:\Users\lucav\Github\Game-Popularity-Prediction-Modelv2\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## 3. Collect Current Data

**Purpose:** Execute the main data collection process.

**Why:** This fetches the latest data from all configured sources (Steam, Twitch, Google Trends, Reddit, Twitter, YouTube).

**IMPORTANT:** Before running this step:
1.  **Check API Keys/Credentials:** Ensure your API keys/secrets/tokens in the `.env` file (located in the project root `c:\\Users\\lucav\\Github\\Game-Popularity-Prediction-Modelv2`) are correct, especially for **Reddit** (Client ID, Secret, User Agent) and **YouTube** (API Key). Incorrect credentials often lead to `401` or `403` errors.
2.  **Check API Quotas/Rate Limits:** APIs like YouTube, Google Trends, and Twitter have usage limits (quotas) and rate limits. If you run this frequently or with many games, you might hit these limits, resulting in `403` (Quota Exceeded) or `429` (Too Many Requests) errors. Check your API provider dashboards (e.g., Google Cloud Console for YouTube) if you encounter persistent errors.

**Expected Output:** 
- Status messages indicating which data sources are being queried.
- Potential warnings or errors if API keys are invalid, quotas are exceeded, or rate limits are hit.
- A DataFrame containing the combined data (if collection is at least partially successful).
- A summary of the collected data shape and columns.

In [3]:
# Collect data for all tracked games
# include_details=True is needed to get game names for Twitch/External lookups
# include_twitch=True fetches Twitch viewership
# include_external=True fetches Google Trends, Reddit, Twitter, YouTube data
print("Starting data collection...")
print("This may take a few minutes depending on the number of games and API responsiveness.")
try:
    current_data_df = collector.collect_current_data(
        include_details=True,
        include_twitch=True,
        include_external=True
    )
    print("\n--- Collected Data Sample ---")
    # Display relevant columns, especially the newly added ones
    display_cols = [
        'app_id', 'name', 'category', 'timestamp', 'player_count', 'twitch_viewer_count',
        'google_trends_avg', 'reddit_subscribers', 'reddit_active_users', 'reddit_recent_posts', 'twitter_recent_count',
        'youtube_total_views', 'youtube_avg_views', 'youtube_avg_likes', 'release_date'
    ]
    display_cols_present = [col for col in display_cols if col in current_data_df.columns]
    display(current_data_df[display_cols_present].head())
    print(f"\nShape: {current_data_df.shape}")
    print("\nColumns:", current_data_df.columns.tolist())
except Exception as e:
    print(f"\nAn error occurred during data collection: {e}")
    # Optionally re-raise if debugging: raise e
    current_data_df = pd.DataFrame() # Ensure df exists but is empty on error

Starting data collection...
This may take a few minutes depending on the number of games and API responsiveness.
[2025-05-06 22:00:41.749294] Collecting data for 20 games...
  Fetching Twitch data for: Forza Horizon 5 (Querying as: 'Forza Horizon 5')
Successfully obtained Twitch Access Token.
    > Twitch ID: 1757732267, Viewers: 530
  Fetching Twitch data for: Kenshi (Querying as: 'Kenshi')
    > Twitch ID: 34025, Viewers: 39
  Fetching Twitch data for: Sea of Thieves: 2025 Edition (Querying as: 'Sea of Thieves')
    > Twitch ID: 490377, Viewers: 1474
  Fetching Twitch data for: NARAKA: BLADEPOINT (Querying as: 'NARAKA: BLADEPOINT')
    > Twitch ID: 515474, Viewers: 402
  Fetching Twitch data for: Rocket League® (Querying as: 'Rocket League')
    > Twitch ID: 30921, Viewers: 6465
  Fetching Twitch data for: PUBG: BATTLEGROUNDS (Querying as: 'PUBG: BATTLEGROUNDS')
    > Twitch ID: 493057, Viewers: 6225
  Fetching Twitch data for: Call of Duty®: Modern Warfare® II (Querying as: 'Call of

c:\Users\lucav\Github\Game-Popularity-Prediction-Modelv2\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\lucav\Github\Game-Popularity-Prediction-Modelv2\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Error fetching Google Trends data for 'Valheim': The request failed: Google returned a response with code 429
    > Google Trends: Failed or no data.
    > Querying Reddit subreddit: r/valheim
    > Reddit Data (Subscribers: 532344, Active: 86, Posts_week: 250)
    > Querying Twitter for: "Valheim" OR #Valheim
Error: Twitter client not initialized.
    > Querying YouTube for: "Valheim" official trailer OR gameplay
    > YouTube Search Query: "Valheim" official trailer OR gameplay
    > YouTube Stats (Top 5 videos): Total Views=1695785, Avg Views=339157, Avg Likes=7867
  Processing external signals for: NARAKA: BLADEPOINT (Timeout: 30s)
    > Querying Google Trends for: 'NARAKA: BLADEPOINT'


c:\Users\lucav\Github\Game-Popularity-Prediction-Modelv2\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\lucav\Github\Game-Popularity-Prediction-Modelv2\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Error fetching Google Trends data for 'NARAKA: BLADEPOINT': The request failed: Google returned a response with code 429
    > Google Trends: Failed or no data.
    > Querying Reddit subreddit: r/NarakaBladePoint
    > Reddit Data (Subscribers: 32179, Active: 6, Posts_week: 40)
    > Querying Twitter for: "NARAKA BLADEPOINT" OR #NARAKABLADEPOINT
Error: Twitter client not initialized.
    > Querying YouTube for: "NARAKA: BLADEPOINT" official trailer OR gameplay
    > YouTube Search Query: "NARAKA: BLADEPOINT" official trailer OR gameplay
    > YouTube Stats (Top 5 videos): Total Views=1556839, Avg Views=311368, Avg Likes=5218
  Processing external signals for: Sea of Thieves: 2025 Edition (Timeout: 30s)
    > Querying Google Trends for: 'Sea of Thieves: 2025 Edition'


...

Error fetching Google Trends data for 'Baldur's Gate 3': The request failed: Google returned a response with code 429
    > Google Trends: Failed or no data.
    > Querying Reddit subreddit: r/BaldursGate3
    > Reddit Data (Subscribers: 3139889, Active: 696, Posts_week: 250)
    > Querying Twitter for: "Baldurs Gate 3" OR #BaldursGate3 OR #BG3
Error: Twitter client not initialized.
    > Querying YouTube for: "Baldurs Gate 3" gameplay OR BG3
    > YouTube Search Query: "Baldurs Gate 3" gameplay OR BG3
    > YouTube Stats (Top 5 videos): Total Views=13362300, Avg Views=2672460, Avg Likes=82632
  Processing external signals for: ELDEN RING (Timeout: 30s)
    > Querying Google Trends for: 'ELDEN RING'


c:\Users\lucav\Github\Game-Popularity-Prediction-Modelv2\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\lucav\Github\Game-Popularity-Prediction-Modelv2\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Error fetching Google Trends data for 'ELDEN RING': The request failed: Google returned a response with code 429
    > Google Trends: Failed or no data.
    > Querying Reddit subreddit: r/Eldenring
    > Reddit Data (Subscribers: 4323617, Active: 755, Posts_week: 250)
    > Querying Twitter for: "ELDEN RING" OR #ELDENRING
Error: Twitter client not initialized.
    > Querying YouTube for: "ELDEN RING" official trailer OR gameplay
    > YouTube Search Query: "ELDEN RING" official trailer OR gameplay
    > YouTube Stats (Top 5 videos): Total Views=32520233, Avg Views=6504047, Avg Likes=150805
[2025-05-06 22:03:13.363554] Finished collecting external signals.
[2025-05-06 22:03:13.363595] Merging external signals into main dataset...
[2025-05-06 22:03:13.363752] Finished merging external signals.
[2025-05-06 22:03:13.404429] Data collection complete. DataFrame shape: (20, 19)

--- Collected Data Sample ---


,app_id,name,category,timestamp,player_count,twitch_viewer_count,google_trends_avg,reddit_subscribers,reddit_active_users,reddit_recent_posts,twitter_recent_count,youtube_total_views,youtube_avg_views,youtube_avg_likes,release_date
0,1551360,Forza Horizon 5,experimental,2025-05-06T22:01:00.908708,10732,530,None,414910.0,65.0,250.0,None,15747930,3149586.0,85406.6,"Nov 8, 2021"
1,233860,Kenshi,declining,2025-05-06T22:01:00.908708,2962,39,None,166219.0,118.0,250.0,None,844159,168831.8,7749.0,"Dec 6, 2018"
2,1172620,Sea of Thieves: 2025 Edition,experimental,2025-05-06T22:01:00.908708,6001,1474,None,NaN,NaN,NaN,None,343551,68710.2,2705.2,"Jun 3, 2020"
3,1203220,NARAKA: BLADEPOINT,experimental,2025-05-06T22:01:00.908708,20303,402,None,32179.0,6.0,40.0,None,1556839,311367.8,5218.4,"Aug 11, 2021"
4,252950,Rocket League®,declining,2025-05-06T22:01:00.908708,19362,6465,None,1833512.0,205.0,250.0,None,4780631,956126.2,20154.6,"Jul 6, 2015"



Shape: (20, 19)

Columns: ['app_id', 'name', 'category', 'timestamp', 'player_count', 'twitch_viewer_count', 'google_trends_avg', 'reddit_subscribers', 'reddit_active_users', 'reddit_recent_posts', 'twitter_recent_count', 'youtube_total_views', 'youtube_avg_views', 'youtube_avg_likes', 'release_date', 'metacritic_score', 'genres', 'price', 'is_free']


## 4. Save Data to File

**Purpose:** Save the collected DataFrame for future use.

**Why:** Persistent storage allows us to build a historical dataset over time, which is essential for the `DataAggregator` and model training.

**Expected Output:** Confirmation of the file save location.

In [4]:
# Save the collected data to a compressed CSV file
try:
    # Check if the DataFrame exists and is not empty
    if 'current_data_df' in locals() and not current_data_df.empty:
        saved_filepath = collector.save_data(data=current_data_df, compress=True)
        print(f"\nData successfully saved to: {saved_filepath}")
        
        # Optional: Verify loading back
        # loaded_data = collector.load_data(saved_filepath)
        # print(f"Verification - Loaded {len(loaded_data)} rows from saved file")
    else:
        print("\nSkipping save: No data collected or collection failed.")
except Exception as e:
     print(f"\nAn error occurred while saving data: {e}")

[2025-05-06 22:04:13.075969] Data saved to ..\data\steam_data_2025-05-06-22-04.csv.gz

Data successfully saved to: ..\data\steam_data_2025-05-06-22-04.csv.gz


## 5. Initial Data Review (Optional)

**Purpose:** Perform a quick check on the collected data.

**Why:** Helps spot any immediate issues like missing values in key columns or unexpected data ranges.

**Expected Output:**
- Basic statistics for key numerical columns.

In [5]:
# Display basic statistics for numerical columns if data was collected
if 'current_data_df' in locals() and not current_data_df.empty:
    print("\nBasic Statistics for Key Metrics:")
    stats_cols = [
        'player_count', 'twitch_viewer_count', 'google_trends_avg', 
        'reddit_subscribers', 'reddit_active_users', 'reddit_recent_posts', 
        'twitter_recent_count', 'youtube_total_views', 'youtube_avg_views', 'youtube_avg_likes'
    ]
    stats_cols_present = [col for col in stats_cols if col in current_data_df.columns]
    display(current_data_df[stats_cols_present].describe())
else:
    print("\nSkipping statistics: No data available.")


Basic Statistics for Key Metrics:


,player_count,twitch_viewer_count,reddit_subscribers,reddit_active_users,reddit_recent_posts,youtube_total_views,youtube_avg_views,youtube_avg_likes
count,20.000000,20.00000,1.700000e+01,17.000000,17.000000,2.000000e+01,2.000000e+01,20.000000
mean,76967.350000,11483.20000,1.632671e+06,259.647059,202.470588,1.608493e+07,3.216986e+06,53915.550000
std,140316.458527,27977.02605,1.275120e+06,235.021792,79.571601,3.144793e+07,6.289586e+06,92843.774347
min,1.000000,39.00000,3.217900e+04,6.000000,40.000000,2.106020e+05,4.212040e+04,426.800000
25%,15131.500000,411.00000,4.149100e+05,65.000000,161.000000,9.558138e+05,1.911628e+05,4252.450000
50%,23271.500000,1908.00000,1.709804e+06,205.000000,250.000000,3.024818e+06,6.049636e+05,13382.400000
75%,78880.750000,6628.25000,2.653057e+06,351.000000,250.000000,1.351990e+07,2.703979e+06,42960.000000
max,617562.000000,124267.00000,4.323617e+06,755.000000,250.000000,1.259228e+08,2.518456e+07,309560.000000


## 6. Next Steps

**Purpose:** Outline the path forward.

**Why:** Guides the project towards the goal of building the prediction model.

**Next Actions:**
1.  **Repeat Collection:** Run this notebook periodically (e.g., daily, weekly) to build up historical data.
2.  **Aggregation:** Once sufficient historical data exists, use the `DataAggregator` (likely in `02_feature_engineering.ipynb`) to process the saved files into a feature set.
3.  **Feature Engineering & Modeling:** Analyze the aggregated data and build predictive models (`02_feature_engineering.ipynb`, `03_modeling.ipynb`).

In [6]:
# Final summary message
print("\nData Collection Notebook Complete.")
if 'saved_filepath' in locals():
    print(f"Latest data saved to: {saved_filepath}")
elif 'current_data_df' in locals() and current_data_df.empty:
     print("Data collection run finished, but resulted in empty data or an error occurred.")
else:
     print("Data collection run finished, but data was not saved (likely due to an error). Check previous cell outputs.")

print("\nRemember to run this notebook periodically to build your historical dataset.")


Data Collection Notebook Complete.
Latest data saved to: ..\data\steam_data_2025-05-06-22-04.csv.gz

Remember to run this notebook periodically to build your historical dataset.


---
*End of Notebook*